In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import copy
import random
import os
import re 
import numpy as np

In [ ]:
class Net(nn.Module):
    def __init__(self, input_dim, n_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, n_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
class StandardDataset(Dataset):
  def __init__(self, data):
    self.data = data

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    unique_id, feature, label = self.data[idx]
    feature = np.float32(feature)
    return (torch.FloatTensor(feature),
            label)

In [ ]:
class SISA:
  def __init__(self, full_data, n_shards, n_slices, model, n_classes):
    """
    full_data: list of n sample <unique_id, sample_feature, sample_label>
    n_shards: integer, number of shards
    n_slices: integer, number of slices
    model: string, architecture to use
    keep_indices: list of k <= n sample of unique ids, use for filter unlearned samples
    """
    self.full_data = full_data
    self.n_shards = n_shards
    self.n_slices = n_slices
    self.n_classes = n_classes
    self.model = model
    self.n_samples = len(full_data)
    self.sample2ss = []
    for i in range(self.n_samples):
      current_shard = random.sample(list(range(self.n_shards)), 1)[0]
      current_slice = random.sample(list(range(self.n_slices)), 1)[0]
      self.sample2ss.append([current_shard, current_slice])   
    self.models =  [[None for j in range(self.n_slices)] for i in range(self.n_shards)]

    #training configurations
    self.batch_size = 16
    self.epochs = 10

  def _update(self, remove_ids):
    #update self.sample2shard, remove_indices -> move to a new class
    #update which shard to be re-trained, including slice step-point
    #compatible to run before _train for unlearning task
    unlearn_shard = self.n_shards + 1 # this shard does not exist before 
    removed_ss = []
    for i in range(self.n_samples):
      unique_id, feature, label = self.full_data[i]
      if(unique_id in remove_ids):
        removed_ss.append(self.sample2ss[i])
        self.sample2ss[i] = [unlearn_shard, 0]

    #figure out which shard to be re-trained / from which slice
    retrain_shards = [False for _ in range(self.n_shards)]
    retrain_slices = [self.n_slices-1 for _ in range(self.n_shards)] 
    for (retrain_shard, retrain_slice) in removed_ss:
      retrain_shards[retrain_shard] = True
      retrain_slices[retrain_shard] = min(retrain_slice, retrain_slices[retrain_shard])
    return retrain_shards, retrain_slices

  def _unlearn(self, retrain_shards, retrain_slices):
    save_path = "results_unlearned/"
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    for i in range(self.n_shards):
      if(retrain_shards[i]):
        starting_slice = retrain_slices[i]
        for j in range(starting_slice, self.n_slices):
          self._train(i, j, self.batch_size, self.epochs, save_path=save_path, device=device)

  def unlearn_do_all(self, remove_ids):
    retrain_shards, retrain_slices = self._update(remove_ids)
    print("Retrain Shards: ", retrain_shards)
    print("Retrain Slices: ", retrain_slices)
    self._unlearn(retrain_shards, retrain_slices)
    print('Finish unlearning ...')

  def learn_do_all(self):
    save_path = "results/"
    device = 'cuda' if torch.cida.is_available() else 'cpu'
    for i in range(self.n_shards):
      for j in range(self.n_slices):
        self._train(i, j, self.batch_size, self.epochs, save_path=save_path, device=device)
    print('Finish learning ...')

  def _train(self, shard_num, slice_num, batch_size, epochs, device="cpu", save_path="results/", verbose=False):
    #shard_num: integer, [0..n_shard-1]
    #slice_num: integer, [0..n_slice-1]

    #step 1: collect data given a shard and slice, put to dataloader
    feature_dim = len(self.full_data[0][1])
    data = []
    for i in range(self.n_samples):
      current_shard, current_slice = self.sample2ss[i]
      if(current_shard == shard_num and current_slice == slice_num):
        data.append(self.full_data[i])
    dataset = StandardDataset(data)
    dataloader = DataLoader(dataset, batch_size=batch_size,shuffle=True)
    
    #step 2: train model
    if(slice_num == 0 or not self.models[shard_num][slice_num-1]):
      model = eval(self.model)(feature_dim ,self.n_classes) #intialize a new model
    else:
      model = self.models[shard_num][slice_num-1] #use previous slice ckpt
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()

    current_step = 0
    for epoch in range(epochs):  
      model.train()
      running_loss = 0.0
      for i, data in enumerate(dataloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if(verbose):
          # print statistics to make sure loss goes down
          running_loss += loss.item()
          current_step += 1
          if (current_step % 200 == 0):    
            print('[%d, %5d] loss: %.3f' %
                  (epoch, current_step, running_loss / 200))
            running_loss = 0.0
    
    #step 3: saving the model
    outname = "sh"+str(shard_num)+"sl"+str(slice_num)+".pt"
    PATH = os.path.join(save_path, outname)
    torch.save(model.state_dict(), PATH)
    #put model to SISA models
    self.models[shard_num][slice_num] = model
    print("Finish training ... Shard: "+ str(shard_num) + " Slice: " + str(slice_num))

In [ ]:
#Sample dataset
import numpy as np
from sklearn.datasets import fetch_kddcup99
data_all = fetch_kddcup99(download_if_missing=True)

features = data_all.data[:, 4:]
labels = data_all.target
print(features[0:5], labels[0:5])

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(labels)

N, D = features.shape
unique_ids = list(range(N))
data = []
for i in range(N):
  data.append([unique_ids[i], features[i], labels[i]])

[[181 5450 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 8 8 0.0 0.0 0.0 0.0 1.0 0.0
  0.0 9 9 1.0 0.0 0.11 0.0 0.0 0.0 0.0 0.0]
 [239 486 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 8 8 0.0 0.0 0.0 0.0 1.0 0.0 0.0
  19 19 1.0 0.0 0.05 0.0 0.0 0.0 0.0 0.0]
 [235 1337 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 8 8 0.0 0.0 0.0 0.0 1.0 0.0
  0.0 29 29 1.0 0.0 0.03 0.0 0.0 0.0 0.0 0.0]
 [219 1337 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 6 6 0.0 0.0 0.0 0.0 1.0 0.0
  0.0 39 39 1.0 0.0 0.03 0.0 0.0 0.0 0.0 0.0]
 [217 2032 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 6 6 0.0 0.0 0.0 0.0 1.0 0.0
  0.0 49 49 1.0 0.0 0.02 0.0 0.0 0.0 0.0 0.0]] [b'normal.' b'normal.' b'normal.' b'normal.' b'normal.']


In [ ]:
n_shards, n_slices = 5, 5
model = "Net"
n_classes = 23
sisa = SISA(data, n_shards, n_slices, model, n_classes)

In [ ]:
#learning on SISA
sisa.learn_do_all()

Finish training ... Shard: 0 Slice: 0
Finish training ... Shard: 0 Slice: 1
Finish training ... Shard: 0 Slice: 2
Finish training ... Shard: 0 Slice: 3
Finish training ... Shard: 0 Slice: 4
Finish training ... Shard: 1 Slice: 0
Finish training ... Shard: 1 Slice: 1
Finish training ... Shard: 1 Slice: 2
Finish training ... Shard: 1 Slice: 3
Finish training ... Shard: 1 Slice: 4
Finish training ... Shard: 2 Slice: 0
Finish training ... Shard: 2 Slice: 1
Finish training ... Shard: 2 Slice: 2
Finish training ... Shard: 2 Slice: 3
Finish training ... Shard: 2 Slice: 4
Finish training ... Shard: 3 Slice: 0
Finish training ... Shard: 3 Slice: 1
Finish training ... Shard: 3 Slice: 2
Finish training ... Shard: 3 Slice: 3
Finish training ... Shard: 3 Slice: 4
Finish training ... Shard: 4 Slice: 0
Finish training ... Shard: 4 Slice: 1
Finish training ... Shard: 4 Slice: 2
Finish training ... Shard: 4 Slice: 3
Finish training ... Shard: 4 Slice: 4
Finish learning ...


In [ ]:
#unlearning on SISA
n_requests = 15
remove_ids = random.sample(unique_ids, n_requests)
sisa.unlearn_do_all(remove_ids)

Retrain Shards:  [True, True, True, True, True]
Retrain Slices:  [4, 1, 0, 1, 1]
Finish training ... Shard: 0 Slice: 4
Finish training ... Shard: 1 Slice: 1
Finish training ... Shard: 1 Slice: 2
Finish training ... Shard: 1 Slice: 3
Finish training ... Shard: 1 Slice: 4
Finish training ... Shard: 2 Slice: 0
Finish training ... Shard: 2 Slice: 1
Finish training ... Shard: 2 Slice: 2
Finish training ... Shard: 2 Slice: 3
Finish training ... Shard: 2 Slice: 4
Finish training ... Shard: 3 Slice: 1
Finish training ... Shard: 3 Slice: 2
Finish training ... Shard: 3 Slice: 3
Finish training ... Shard: 3 Slice: 4
Finish training ... Shard: 4 Slice: 1
Finish training ... Shard: 4 Slice: 2
Finish training ... Shard: 4 Slice: 3
Finish training ... Shard: 4 Slice: 4
Finish unlearning ...


In [ ]:
class SISA_inference:
  """
  this class do the "Aggregation" in SISA
  majority voting and return accuracy
  """
  def __init__(self, test_data, n_shards, n_slices, model, n_classes, learning_path, unlearning_path=None):
    """
    full_data: list of n sample <unique_id, sample_feature, sample_label>
    if unlearning_path is None, inference before unlearning
    if unlearning path is None, inference after unlearning
    """
    self.test_data = test_data
    self.n_shards = n_shards
    self.n_slices = n_slices
    self.n_classes = n_classes
    self.model = model
    self.n_samples = len(test_data) 
    self.feature_dim = len(self.test_data[0][1])
    self.batch_size = 8
    self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    #loading models' checkpoints
    self.models =  [None for _ in range(self.n_shards)]
    if(unlearning_path):
      for model_name in os.listdir(unlearning_path):
        current_shard, current_slice = re.findall(r'\d+', model_name)
        #if not the last slice, skip the model
        if(int(current_shard) >= self.n_shards or int(current_slice) < self.n_slices - 1):
          continue
        saved_path = os.path.join(unlearning_path, model_name)
        model = eval(self.model)(self.feature_dim ,self.n_classes)
        model.load_state_dict(torch.load(saved_path))
        model.to(self.device)
        model.eval()
        self.models[int(current_shard)] = model

    #load learning models
    for model_name in os.listdir(learning_path):
      current_shard, current_slice = re.findall(r'\d+', model_name)
      if(int(current_shard) >= self.n_shards or self.models[int(current_shard)]): #loaded unlearned models
        continue 
      #if not the last slice, skip the model
      if(int(current_slice) < self.n_slices - 1):
        continue
      saved_path = os.path.join(learning_path, model_name)
      model = eval(self.model)(self.feature_dim ,self.n_classes)
      model.load_state_dict(torch.load(saved_path))
      model.to(self.device)
      model.eval()
      self.models[int(current_shard)] = model  


    #assert that all models are loaded
    assert None not in self.models
    print("All contituent models loaded ...")

  def inference(self):
    #create dataloader
    dataset = StandardDataset(self.test_data)
    dataloader = DataLoader(dataset, batch_size=self.batch_size,shuffle=False)

    true_labels, predicted_labels = [], []
    with torch.no_grad():
      for i, data in enumerate(dataloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(self.device), labels.to(self.device)
        B = inputs.size(0)
        outputs = torch.zeros((B, self.n_classes), device=self.device)
        for j in range(self.n_shards):
            outputs += self.models[j](inputs)
        y_pred = list(outputs.argmax(dim=-1).detach().cpu().numpy())
        y_true = list(labels.detach().cpu().numpy())
        
        predicted_labels += y_pred
        true_labels += y_true
    return true_labels, predicted_labels

In [ ]:
#prediction on the original trained models (no unlearning done)
#prediction on the training set (did not split train/test before)
sisa_inference = SISA_inference(test_data=data,
                                n_shards=5,
                                n_slices=5,
                                model="Net",
                                n_classes=n_classes,
                                learning_path = "results/")

All contituent models loaded ...


In [ ]:
from sklearn.metrics import accuracy_score
y_true, y_pred = sisa_inference.inference()
print("Accuracy Score: ", accuracy_score(y_true, y_pred))

Accuracy Score:  0.9946459765880398


In [ ]:
#prediction on the original unlearned models (no unlearning done)
#prediction on the training set (did not split train/test before)
sisa_inference = SISA_inference(test_data=data,
                                n_shards=5,
                                n_slices=5,
                                model="Net",
                                n_classes=n_classes,
                                learning_path = "results/",
                                unlearning_path = "results_unlearned/")

All contituent models loaded ...


In [ ]:
from sklearn.metrics import accuracy_score
y_true, y_pred = sisa_inference.inference()
print("Accuracy Score: ", accuracy_score(y_true, y_pred))

Accuracy Score:  0.9957775074338945


In [ ]:
#Baseline training and re-training from scratch
#set num_shards = 1 and num_slices = 1
n_shards, n_slices = 1, 1
model = "Net"
n_classes = 23
sisa_baseline = SISA(data, n_shards, n_slices, model, n_classes)

In [ ]:
#learning on SISA baseline
sisa_baseline.learn_do_all()

Finish training ... Shard: 0 Slice: 0
Finish learning ...


In [ ]:
#unlearning on SISA baseline, same remove_ids as SISA from above
sisa_baseline.unlearn_do_all(remove_ids)

Retrain Shards:  [True]
Retrain Slices:  [0]
Finish training ... Shard: 0 Slice: 0
Finish unlearning ...


In [ ]:
sisa_inference_baseline = SISA_inference(test_data=data,
                                n_shards=1,
                                n_slices=1,
                                model="Net",
                                n_classes=n_classes,
                                learning_path = "results/")

from sklearn.metrics import accuracy_score
y_true, y_pred = sisa_inference_baseline.inference()
print("Accuracy Score: ", accuracy_score(y_true, y_pred))

All contituent models loaded ...
Accuracy Score:  0.9949273411454169


In [ ]:
sisa_inference_baseline = SISA_inference(test_data=data,
                                n_shards=1,
                                n_slices=1,
                                model="Net",
                                n_classes=n_classes,
                                learning_path = "results/",
                                unlearning_path = "results_unlearned/")

from sklearn.metrics import accuracy_score
y_true, y_pred = sisa_inference_baseline.inference()
print("Accuracy Score: ", accuracy_score(y_true, y_pred))

All contituent models loaded ...
Accuracy Score:  0.9977652771845731
